In [2]:
!pip install seqeval
!pip install sklearn_crfsuite
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=9cc318ebb38eea7d12e0b36ec31facd77ee9b8839eba930f57d31c8fa2b4b282
  Stored in directory: /root/.cache/pip/wheels/b2/a1/b7/0d3b008d0c77cd57332d724b92cf7650b4185b493dc785f00a
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.9/966.9 kB 15.1 MB/s eta 0:00:00a 0:00:01
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-24awcyap
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-24awcyap
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101076 sha256=f0362dafd26ce0125bfa

In [3]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from prettytable import PrettyTable
import nltk
from nltk.corpus import stopwords
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import warnings
warnings.filterwarnings('ignore')
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from itertools import chain
import shutil
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
import pickle
import random
from sklearn.model_selection import RandomizedSearchCV

In [6]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /usr/share/nltk_data...
[nltk_data]    | Downloading pa

True

In [7]:
import json

In [8]:
# Read data from JSON file
with open('/kaggle/input/hackathon-online-nlu-slot-filling2/train.json') as json_file: 
    data_dict = json.load(json_file) 
    
with open('/kaggle/input/hackathon-online-nlu-slot-filling2/valid.json') as json_file: 
    vdata_dict = json.load(json_file)

In [9]:
train_utterance = []
train_slots = []

val_utterance = []
val_slots = []
# Load data into respective dicts
for data in data_dict:
    train_utterance.append(data['utterance'])
    train_slots.append(data['slots'])
    
for vdata in vdata_dict:
    val_utterance.append(vdata['utterance'])
    val_slots.append(vdata['slots'])

In [10]:
#train_utterance

In [11]:
#train_slots

In [12]:
len(train_utterance),len(train_slots)

(13084, 13084)

In [13]:
len(val_utterance),len(val_slots)

(200, 200)

In [14]:
STOP_WORDS = stopwords.words('english')

def word2features(sent, i):
    sent = sent.split(" ")
    word = sent[i]
    global STOP_WORDS

    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'word.length':len(word),
        'word.stop_word':word.lower() in STOP_WORDS,
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:word.length':len(word1),
            '-1:word.stop_word':word1.lower() in STOP_WORDS
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:word.length':len(word1),
            '+1:word.stop_words':word1.lower() in STOP_WORDS
        })
    else:
        features['EOS'] = True
    return features

In [15]:
def doword2features(sent):
    return [word2features(sent, i) for i in range(len(sent.split()))]

def labels(sent):
    return [label for label in sent.split()] 

In [16]:
train_x = [doword2features(s) for s in train_utterance]
train_y = [labels(s) for s in train_slots]
val_x = [doword2features(s) for s in val_utterance]
val_y = [labels(s) for s in val_slots]

In [17]:
len(train_x), len(train_y), len(val_x), len(val_y)

(13084, 13084, 200, 200)

In [18]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=250,
    all_possible_transitions=True
)

try:
    crf.fit(train_x, train_y)
except AttributeError:
    pass

In [19]:
predicted = crf.predict(val_x)

In [20]:
df = pd.read_csv('/kaggle/input/hackathon-online-nlu-slot-filling2/test.csv')
df.head(10)

,id,utterance
0,1,add this tune to my playlist guest list mashable
1,2,i d like to listen to the song the natural farmer
2,3,i need a table for 8 people at a restaurant in...
3,4,open the canciones del recuerdo playlist and p...
4,5,i want to hear something from the top-fifty by...
5,6,i rate the mathematical magpie chronicle a 0 of 6
6,7,show the movie schedule of animated movies clo...
7,8,what time is careful he might hear you playin...
8,9,i m looking to get a seat at a brasserie in togo
9,10,i need a table for 8 during midday in montana


In [21]:
#df = df.dropna()

In [22]:
test_utterance = [doword2features(s) for s in df['utterance']]

In [23]:
predicted_test = crf.predict(test_utterance)

In [24]:
#for i in range(2544):
    #predicted_sub.append('K')

In [25]:
#predicted_test

In [26]:
import csv

header = ['id_word', 'slots']  # define the header row

with open('submission.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for i, row in enumerate(predicted_test):
        for j, value in enumerate(row):
            writer.writerow([f"{i+1}_{j+1}", value])